In [1]:
import sys
sys.path.append("../..")

import dataInterpreter as dt
import enrichmentAnalysis as ea
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
pd.options.mode.chained_assignment = None  # default='warn'

path = "C:\\Users\\Work\\Desktop\\bicpams_5.1\\data\\latecovid\\"
results_path = "C:\\Users\\Work\\Desktop\\MasterThesis\\Biclustering\\Results\\"
#results_path = "Results/"

In [2]:
filtered_data_01 = pd.read_csv(path + 'data-p01.csv', index_col = 0, sep = '\t')
filtered_data_05 = pd.read_csv(path + 'data-p05.csv', index_col = 0, sep = '\t')

top200 = pd.read_csv(path + 'data-top200.csv', index_col = 0, sep = '\t')
top1000 = pd.read_csv(path + 'data-top1000.csv', index_col = 0, sep = '\t')
top5000 = pd.read_csv(path + 'data-top5000.csv', index_col = 0, sep = '\t')

In [ ]:
def binary_search_results(data, value, data_selector_lambda = lambda x, i: x[i]):
    start = 0
    end = len(data) - 1
    
    if data_selector_lambda(data, 0) >= value:
        return 0
    
    # detect a boundary in an ordered vector
    while start < end:
        i = int((end + start) / 2)
        
        if data_selector_lambda(data, i) < value and data_selector_lambda(data, i + 1) >= value:
            break
        
        if data_selector_lambda(data, i) >= value:
            # go left
            end = i - 1
        else:
            # go right
            start = i + 1
            
    i = int((end + start) / 2)
    return i + 1

In [ ]:
import re

def import_bicPAM(filename):
    file = open(filename, 'r')
    
    # get number of biclusters found
    num_biclusters = int(re.search("[0-9]+", file.readline()).group())
    
    rx = re.compile('[-+]?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?', re.VERBOSE)
    
    # read summary lines
    for i in range(num_biclusters):
        bic_info = file.readline() #.split(' ')
        #print(bic_info[3][3:-1].split(','))
    
    file.readline()
    
    result = []
    
    for i in range(num_biclusters):
        bic_info = file.readline().split(' ')
        size = eval(bic_info[2])
        conditions = bic_info[3][3:-1].split(',')
        pvalue = float(bic_info[6].split('=')[1])
        
        genes = []
        for j in range(size[1]):
            genes.append(file.readline().split('\t')[0])
            
        enriched_terms = ea.getEnrichment(genes, 'GO_Biological_Process_2021')['GO_Biological_Process_2021']
        
        bic = {'size': size, 'conditions': conditions, 'genes': genes, 'pvalue': pvalue, 'enriched_terms': enriched_terms}
        result.append(bic)
        
        file.readline()
        file.readline()
    
    return result
    

In [ ]:
res = import_bicPAM(results_path + 'result-p05-bicPAM.txt')
res

In [ ]:
def import_bics_biclust(filename):
    file = open(filename, 'r')
    
    # Read line of algorithm parameters
    line = file.readline()
    
    result = []
    
    while line != '':
        # size line, gene line, conditions
        for i in range(3):
            size_text = file.readline().split(' ')
            if len(size_text) != 2: 
                return result
            size = (int(size_text[1]), int(size_text[0]))
            
            genes = file.readline()[:-1].split(' ')
            conditions = file.readline()[:-1].split(' ')
            
            enriched_terms = ea.getEnrichment(list(genes), 'GO_Biological_Process_2021')['GO_Biological_Process_2021']
            
            bic = {'size': size, 'conditions': conditions, 'genes': genes, 'enriched_terms': enriched_terms}
            result.append(bic)
    
    # usually shouldn't reach this
    return result

In [ ]:
bicpam_p01[0]['size']

In [ ]:
results = {}

results['bicpam_p01'] = import_bicPAM(results_path + 'result-p01-bicPAM.txt')
results['bicpam_p05'] = import_bicPAM(results_path + 'result-p05-bicPAM.txt')
results['bicpam_anova200'] = import_bicPAM(results_path + 'result-top200-bicPAM.txt')
results['bicpam_anova1000'] = import_bicPAM(results_path + 'result-top1000-bicPAM.txt')
results['bicpam_anova5000'] = import_bicPAM(results_path + 'result-top5000-bicPAM.txt')

results['cheng_church_p01'] = import_bics_biclust(results_path + 'results-p01-chengAndChurch.txt')
results['cheng_church_p05'] = import_bics_biclust(results_path + 'results-p05-chengAndChurch.txt')
results['cheng_church_anova200'] = import_bics_biclust(results_path + 'results-top200-chengAndChurch.txt')
results['cheng_church_anova1000'] = import_bics_biclust(results_path + 'results-top1000-chengAndChurch.txt')
results['cheng_church_anova5000'] = import_bics_biclust(results_path + 'results-top5000-chengAndChurch.txt')

results['plaid_p01'] = import_bics_biclust(results_path + 'results-p01-plaid.txt')
results['plaid_p05'] = import_bics_biclust(results_path + 'results-p05-plaid.txt')
results['plaid_anova200'] = import_bics_biclust(results_path + 'results-top200-plaid.txt')
results['plaid_anova1000'] = import_bics_biclust(results_path + 'results-top1000-plaid.txt')
results['plaid_anova5000'] = import_bics_biclust(results_path + 'results-top5000-plaid.txt')

results['xMotifs_p01'] = import_bics_biclust(results_path + 'results-p01-xMotifs.txt')
results['xMotifs_p05'] = import_bics_biclust(results_path + 'results-p05-xMotifs.txt')
results['xMotifs_anova200'] = import_bics_biclust(results_path + 'results-top200-xMotifs.txt')
results['xMotifs_anova1000'] = import_bics_biclust(results_path + 'results-top1000-xMotifs.txt')
results['xMotifs_anova5000'] = import_bics_biclust(results_path + 'results-top5000-xMotifs.txt')

In [ ]:
import json

#with open('processed_results.json', 'w') as file:
     #file.write(json.dumps(results)) # use `json.loads` to do the reverse

In [3]:
import json

with open('processed_results.json') as file:
    results = json.load(file)

In [4]:
def prettify_biclust_method(method):
    if method == 'bicpam':
        return 'BicPAM'
    elif method == 'cheng_church':
        return 'Cheng and Church'
    elif method == 'plaid':
        return 'Plaid'
    else:
        return method

def prettify_gene_select_method(method):
    if method.startswith('p'):
        return 'p-value < 0.' + method.rpartition('p')[-1] + ' (Mann-Whitney U test)'
    elif method.startswith('anova'):
        return 'ANOVA, top ' + method.rpartition('anova')[-1] + ' features'

In [49]:
biclusters = []

for method in results:
    method_info = method.rpartition('_')
    biclust_method = prettify_biclust_method(method_info[0])
    gene_selection_method = prettify_gene_select_method(method_info[2])
    
    for bic in results[method]:
        dataset = {'index': [], 'p-value': [], 'c-score': []}

        for term in bic['enriched_terms']:
            if term[6] >= 0.05:
                break
            
            dataset['index'] += [term[1]]
            dataset['p-value'] += [term[6]]
            dataset['c-score'] += [term[4]]
                
        #'size': size, 'conditions': conditions, 'genes': genes, 'pvalue': pvalue, 'enriched_terms': enriched_terms
        
        bicluster = {'size': bic['size'], 'genes': bic['genes'], 'conditions': bic['conditions'], 'terms': dataset, 'info': {'algorithm': biclust_method, 'data_selection': gene_selection_method, 'statistic': (bic['pvalue'] if biclust_method == 'BicPAM' else None)}}
        biclusters.append(bicluster)

In [ ]:
import json

#with open('biclusters.json', 'w') as file:
#     file.write(json.dumps(biclusters)) # use `json.loads` to do the reverse

In [ ]:
import json

with open('biclusters.json') as file:
    biclusters = json.load(file)

In [ ]:
for bic in biclusters:
    print(bic['conditions'])

In [6]:
index = {}

cell_types = set()
conditions = set()

for biclust in biclusters: 
    for cond in biclust['conditions']:
        info = dt.get_info_from_name(cond)

        if info['Cell Type'] not in index:
            index[info['Cell Type']] = set()

        index[info['Cell Type']].add(info['Condition'])

multiindex = []

for key in index:
    for subkey in index[key]:
        multiindex.append((key, subkey))

In [7]:
def find_in_multiindex(e0, e1, multiindex):
    i = 0
    for e in multiindex:
        if e[0] == e0 and e[1] == e1:
            return i
        
        i += 1

In [8]:
[(x[0] + ' ' + x[1]) for x in multiindex]

['A549 HPIV3',
 'A549 IAV',
 'A549 RSV',
 'A549 SARS-CoV-2',
 'A549 Healthy',
 'NHBE IAV',
 'NHBE IAVdNS1',
 'NHBE SARS-CoV-2',
 'NHBE Healthy',
 'Calu3 SARS-CoV-2',
 'Calu3 Healthy',
 'Biopsy SARS-CoV-2',
 'Biopsy Healthy']

In [50]:
index = [(x[0] + ' ' + x[1]) for x in multiindex]
index_counts = [(x[0] + ' ' + x[1] + ' count') for x in multiindex]
index_p = [(x[0] + ' ' + x[1] + ' p') for x in multiindex]

joined_terms = {}
joined_terms_counts = {}
joined_terms_p = {}

for bic in biclusters:
    num_conditions = len([c for c in bic['conditions'] if dt.get_info_from_name(c)['Condition'] != 'Healthy'])
    term_names = bic['terms']['index']
    c_scores = bic['terms']['c-score']
    pvalues = bic['terms']['p-value']
    
    for term_id in range(len(term_names)):
        if term_names[term_id] not in joined_terms:
            joined_terms[term_names[term_id]] = [0] * len(index)
            joined_terms_counts[term_names[term_id]] = [0] * len(index)
            joined_terms_p[term_names[term_id]] = [1] * len(index)
        
        for cond in bic['conditions']:
            info = dt.get_info_from_name(cond)
            if info['Condition'] == 'Healthy':
                continue
                
            prev = joined_terms[term_names[term_id]][index.index(info['Cell Type'] + ' ' + info['Condition'])]
            
            if c_scores[term_id] > prev:
                joined_terms[term_names[term_id]][index.index(info['Cell Type'] + ' ' + info['Condition'])] = c_scores[term_id]
                
            prev_p = joined_terms[term_names[term_id]][index.index(info['Cell Type'] + ' ' + info['Condition'])]
            if pvalues[term_id] < prev_p:
                joined_terms_p[term_names[term_id]][index.index(info['Cell Type'] + ' ' + info['Condition'])] = pvalues[term_id]
            
            joined_terms_counts[term_names[term_id]][index.index(info['Cell Type'] + ' ' + info['Condition'])] += 1 #/ num_conditions

In [51]:
no_healthy_index = [i for i in index if 'Healthy' not in i]

results_c = pd.DataFrame(joined_terms, index = index).T[no_healthy_index]
results_counts = pd.DataFrame(joined_terms_counts, index = index).T[no_healthy_index]
results_p = pd.DataFrame(joined_terms_p, index = index).T[no_healthy_index]

In [52]:
joined_results = pd.concat([results_counts, results_c, results_p], axis=1, keys=['count', 'c-score', 'p-value']).swaplevel(0,1,axis=1).sort_index(axis=1)
joined_results

A549 HPIV3        \
                                                        c-score count   
cytokine-mediated signaling pathway (GO:0019221)     440.129005    73   
collagen fibril organization (GO:0030199)            100.291072     9   
extracellular matrix organization (GO:0030198)        84.608782    38   
interferon-gamma-mediated signaling pathway (GO...   972.372282    53   
cellular response to cytokine stimulus (GO:0071...   148.519013    37   
...                                                         ...   ...   
long-term synaptic potentiation (GO:0060291)           0.000000     0   
calcium ion transmembrane transport (GO:0070588)       0.000000     0   
transcription initiation from RNA polymerase II...     0.000000     0   
dolichyl diphosphate biosynthetic process (GO:0...  2309.985134     1   
dolichyl diphosphate metabolic process (GO:0046...  2309.985134     1   

                                                                A549 IAV  \
                                                     p-value     c-score   
cytokine-mediated signaling pathway (GO:0019221)    0.006953  436.200458   
collagen fibril organization (GO:0030199)           0.021609   89.049709   
extracellular matrix organization (GO:0030198)      0.012892   42.334989   
interferon-gamma-mediated signaling pathway (GO...  0.001027  386.908645   
cellular response to cytokine stimulus (GO:0071...  0.025862  235.261251   
...                                                      ...         ...   
long-term synaptic potentiation (GO:0060291)        1.000000    0.000000   
calcium ion transmembrane transport (GO:0070588)    1.000000    0.000000   
transcription initiation from RNA polymerase II...  1.000000    0.000000   
dolichyl diphosphate biosynthetic process (GO:0...  0.000613    0.000000   
dolichyl diphosphate metabolic process (GO:0046...  0.000613    0.000000   

                                                                    \
                                                   count   p-value   
cytokine-mediated signaling pathway (GO:0019221)      69  0.037165   
collagen fibril organization (GO:0030199)              3  0.013519   
extracellular matrix organization (GO:0030198)        34  0.028781   
interferon-gamma-mediated signaling pathway (GO...    40  0.008734   
cellular response to cytokine stimulus (GO:0071...    48  0.025862   
...                                                  ...       ...   
long-term synaptic potentiation (GO:0060291)           0  1.000000   
calcium ion transmembrane transport (GO:0070588)       0  1.000000   
transcription initiation from RNA polymerase II...     0  1.000000   
dolichyl diphosphate biosynthetic process (GO:0...     0  1.000000   
dolichyl diphosphate metabolic process (GO:0046...     0  1.000000   

                                                      A549 RSV        \
                                                       c-score count   
cytokine-mediated signaling pathway (GO:0019221)     95.735451    45   
collagen fibril organization (GO:0030199)             0.000000     0   
extracellular matrix organization (GO:0030198)       39.799158    21   
interferon-gamma-mediated signaling pathway (GO...  386.908645    32   
cellular response to cytokine stimulus (GO:0071...   83.639639    23   
...                                                        ...   ...   
long-term synaptic potentiation (GO:0060291)          0.000000     0   
calcium ion transmembrane transport (GO:0070588)      0.000000     0   
transcription initiation from RNA polymerase II...    0.000000     0   
dolichyl diphosphate biosynthetic process (GO:0...    0.000000     0   
dolichyl diphosphate metabolic process (GO:0046...    0.000000     0   

                                                             A549 SARS-CoV-2  \
                                                     p-value         c-score   
cytokine-mediated signaling pathway (GO:0019221)    0.006953   605857.804603   
collagen fibril organizati

In [56]:
joined_results[('NHBE SARS-CoV-2', 'p-value')][joined_results[('NHBE SARS-CoV-2', 'p-value')] < 0.001]

cytokine-mediated signaling pathway (GO:0019221)                                   0.000279
interferon-gamma-mediated signaling pathway (GO:0060333)                           0.000963
cellular response to cytokine stimulus (GO:0071345)                                0.000250
negative regulation of intracellular signal transduction (GO:1902532)              0.000009
supramolecular fiber organization (GO:0097435)                                     0.000941
                                                                                     ...   
peptidyl-tyrosine phosphorylation (GO:0018108)                                     0.000957
heme catabolic process (GO:0042167)                                                0.000586
porphyrin-containing compound catabolic process (GO:0006787)                       0.000586
exocyst localization (GO:0051601)                                                  0.000500
negative regulation of antigen receptor-mediated signaling pathway (GO:0050858) 

In [13]:
len(np.unique(joined_results.index))

4440

In [16]:
import itertools

column_names = list(itertools.product(no_healthy_index, ['count', 'p-value', 'c-score']))

covid_index = [x for x in column_names if 'SARS-CoV-2' in x[0]]
a549_index = [x for x in column_names if 'A549' in x[0]]
nhbe_index = [x for x in column_names if 'NHBE' in x[0]]

In [17]:
a549_index

[('A549 HPIV3', 'count'),
 ('A549 HPIV3', 'p-value'),
 ('A549 HPIV3', 'c-score'),
 ('A549 IAV', 'count'),
 ('A549 IAV', 'p-value'),
 ('A549 IAV', 'c-score'),
 ('A549 RSV', 'count'),
 ('A549 RSV', 'p-value'),
 ('A549 RSV', 'c-score'),
 ('A549 SARS-CoV-2', 'count'),
 ('A549 SARS-CoV-2', 'p-value'),
 ('A549 SARS-CoV-2', 'c-score')]

In [ ]:
joined_results.sort_values(('NHBE SARS-CoV-2', 'count'), ascending = False).head(200).loc[:, covid_index]#.sort_values(('NHBE SARS-CoV-2', 'c-score'), ascending = False).head(5).loc[:, covid_index]

In [ ]:
list(set([x[0] for x in covid_index]))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 6))
mask = np.triu(np.ones_like(results_c.corr(method = 'spearman'), dtype=bool))
sns.heatmap(results_c.corr(method = 'spearman'), vmin = -1, vmax = 1, annot = True, cmap='BrBG', mask = mask)

# SARS-CoV-2 Comparison

In [14]:
rank_dataframe = pd.DataFrame(joined_results.loc[:, ('A549 SARS-CoV-2', 'count')].rank(ascending = False).values, index = joined_results.index, columns = ['A549']) #

rank_dataframe['NHBE'] = joined_results.loc[:, ('NHBE SARS-CoV-2', 'count')].rank(ascending = False)
rank_dataframe['Calu3'] = joined_results.loc[:, ('Calu3 SARS-CoV-2', 'count')].rank(ascending = False)
rank_dataframe['Biopsy'] = joined_results.loc[:, ('Biopsy SARS-CoV-2', 'count')].rank(ascending = False)

rank_dataframe.insert(len(rank_dataframe.columns), "prod", rank_dataframe.prod(axis = 1))

rank_dataframe.sort_values('prod').head(25)

,A549,NHBE,Calu3,Biopsy,prod
cytokine-mediated signaling pathway (GO:0019221),1.0,1.5,1.5,52.5,1.181250e+02
regulation of immune response (GO:0050776),15.5,9.0,1.5,7.0,1.464750e+03
cellular response to cytokine stimulus (GO:0071345),3.0,1.5,3.5,153.5,2.417625e+03
cellular response to interferon-gamma (GO:0071346),2.0,6.5,3.5,91.5,4.163250e+03
positive regulation of response to external stimulus (GO:0032103),8.0,6.5,5.0,45.5,1.183000e+04
inflammatory response (GO:0006954),10.5,32.5,10.5,12.5,4.478906e+04
interferon-gamma-mediated signaling pathway (GO:0060333),4.0,17.0,8.0,582.0,3.166080e+05
protein modification by small protein removal (GO:0070646),13.0,80.0,8.0,60.5,5.033600e+05
mRNA processing (GO:0006397),24.5,20.5,34.0,45.5,7.769808e+05
neutrophil activation involved in immune response (GO:0002283),5.5,17.0,34.0,340.0,1.080860e+06


In [29]:
selection = joined_results.loc[rank_dataframe.sort_values('prod').index, covid_index]

for i in covid_index:
    if i[1] == 'p-value':
        selection.loc[:, i] = selection.loc[:, i].map(lambda x: '%.2E' % x)
    elif i[1] == 'c-score':
        selection.loc[:, i] = selection.loc[:, i].map(lambda x: '%.2E' % x)
    elif i[1] == 'count':
        selection.loc[:, i] = (selection.loc[:, i] - selection.loc[:, i].min()) / (selection.loc[:, i].max() - selection.loc[:, i].min())
        selection.loc[:, i] = selection.loc[:, i].map(lambda x: '%.2f' % x)

selection = selection.head(25)

selection.loc[:, [x for x in covid_index if ('A549' in x[0] or 'NHBE' in x[0])]].to_csv('../sars_comparison_p1.csv')
selection.loc[:, [x for x in covid_index if ('Calu3' in x[0] or 'Biopsy' in x[0])]].to_csv('../sars_comparison_p2.csv')

selection

array(['6.95E-03', '3.64E-07', '2.50E-04', ..., '1.00E+00', '1.00E+00',
       '1.00E+00'], dtype=object)

# A549 Virus Comparison

In [ ]:
rank_dataframe = pd.DataFrame(joined_results.loc[:, ('A549 SARS-CoV-2', 'count')].rank(ascending = False).values, index = joined_results.index, columns = ['SARS-CoV-2']) #

rank_dataframe['IAV'] = joined_results.loc[:, ('A549 IAV', 'count')].rank(ascending = False)
rank_dataframe['HPIV3'] = joined_results.loc[:, ('A549 HPIV3', 'count')].rank(ascending = False)
rank_dataframe['RSV'] = joined_results.loc[:, ('A549 RSV', 'count')].rank(ascending = False)

rank_dataframe.insert(len(rank_dataframe.columns), 'prod', rank_dataframe.mean(axis = 1))

rank_dataframe.sort_values('prod').head(25)

In [ ]:
selection = joined_results.loc[rank_dataframe.sort_values('prod').index, a549_index]

for i in a549_index:
    if i[1] == 'p-value':
        selection.loc[:, i] = selection.loc[:, i].map(lambda x: '%.2E' % x)
    elif i[1] == 'c-score':
        selection.loc[:, i] = selection.loc[:, i].map(lambda x: '%.2E' % x)
    elif i[1] == 'count':
        selection.loc[:, i] = (selection.loc[:, i] - selection.loc[:, i].min()) / (selection.loc[:, i].max() - selection.loc[:, i].min())
        selection.loc[:, i] = selection.loc[:, i].map(lambda x: '%.2f' % x)

        
selection = selection.head(25)

selection.loc[:, [x for x in a549_index if ('SARS-CoV-2' in x[0] or 'RSV' in x[0])]].to_csv('../a549_comparison_p1.csv')
selection.loc[:, [x for x in a549_index if ('HPIV3' in x[0] or 'IAV' in x[0])]].to_csv('../a549_comparison_p2.csv')
        
selection

# NHBE Virus Comparison

In [ ]:
rank_dataframe = pd.DataFrame(joined_results.loc[:, ('NHBE SARS-CoV-2', 'count')].rank(ascending = False).values, index = joined_results.index, columns = ['SARS-CoV-2']) #

rank_dataframe['IAV'] = joined_results.loc[:, ('NHBE IAV', 'count')].rank(ascending = False)
rank_dataframe['IAVdNS1'] = joined_results.loc[:, ('NHBE IAVdNS1', 'count')].rank(ascending = False)

rank_dataframe.insert(len(rank_dataframe.columns), 'prod', rank_dataframe.mean(axis = 1))

rank_dataframe.sort_values('prod').head(25)

In [ ]:
selection = joined_results.loc[rank_dataframe.sort_values('prod').index, nhbe_index]

for i in nhbe_index:
    if i[1] == 'p-value':
        selection.loc[:, i] = selection.loc[:, i].map(lambda x: '%.2E' % x)
    elif i[1] == 'c-score':
        selection.loc[:, i] = selection.loc[:, i].map(lambda x: '%.2E' % x)
    elif i[1] == 'count':
        selection.loc[:, i] = (selection.loc[:, i] - selection.loc[:, i].min()) / (selection.loc[:, i].max() - selection.loc[:, i].min())
        selection.loc[:, i] = selection.loc[:, i].map(lambda x: '%.2f' % x)

selection = selection.head(25)

selection.loc[:, [x for x in nhbe_index if ('IAVdNS1' not in x[0])]].to_csv('../nhbe_comparison_p1.csv')
selection.loc[:, [x for x in nhbe_index if ('IAVdNS1' in x[0])]].to_csv('../nhbe_comparison_p2.csv')

selection

# Statistics

In [ ]:
new_multiindex = []

for i in index:
    new_multiindex += (i, 'c-score')

In [ ]:
results_p

In [ ]:
table1 = {'Number of Biclusters': [], 'Average Number of Genes per Bicluster': [], 'Standard Deviation of Genes per Bicluster': [], 'Average Number of Conditions per Bicluster': [], 'Standard Deviation of Conditions per Bicluster': [], 'Average of Enriched Terms per Bicluster': []}
table1_index = []

for method in results:
    method_info = method.rpartition('_')
    
    table1_index.append((prettify_biclust_method(method_info[0]), prettify_gene_select_method(method_info[2])))
    table1['Number of Biclusters'].append(len(results[method]))
    table1['Average Number of Genes per Bicluster'].append(np.mean([x['size'][1] for x in results[method]]))
    table1['Standard Deviation of Genes per Bicluster'].append(np.std([x['size'][1] for x in results[method]]))
    table1['Average Number of Conditions per Bicluster'].append(np.mean([x['size'][0] for x in results[method]]))
    table1['Standard Deviation of Conditions per Bicluster'].append(np.std([x['size'][0] for x in results[method]]))
    table1['Average of Enriched Terms per Bicluster'].append(np.mean([binary_search_results(x['enriched_terms'], 0.01, lambda x, i: x[i][6]) for x in results[method]]))

In [ ]:
statistics_dataset = pd.DataFrame(table1, pd.MultiIndex.from_tuples(table1_index))

pd.set_option("display.max_rows", None)

statistics_dataset['Average Number of Genes per Bicluster'] = statistics_dataset['Average Number of Genes per Bicluster'].map(lambda x: '%.2f' % x)
statistics_dataset['Standard Deviation of Genes per Bicluster'] = statistics_dataset['Standard Deviation of Genes per Bicluster'].map(lambda x: '%.2f' % x)
statistics_dataset['Average Number of Conditions per Bicluster'] = statistics_dataset['Average Number of Conditions per Bicluster'].map(lambda x: '%.2f' % x)
statistics_dataset['Standard Deviation of Conditions per Bicluster'] = statistics_dataset['Standard Deviation of Conditions per Bicluster'].map(lambda x: '%.2f' % x)
statistics_dataset['Average of Enriched Terms per Bicluster'] = statistics_dataset['Average of Enriched Terms per Bicluster'].map(lambda x: '%.2f' % x)

statistics_dataset.to_csv('biclustering_statistics.csv')
statistics_dataset

In [ ]:
statistics_dataset.to_csv(results_path + 'biclustering_statistics.csv')